In [1]:
import operator
import re
import sys

In [2]:
from movie import Movie

In [3]:
episode_re = re.compile("\(\d\d\d\d\)\s\{") # FIX: this does not match if the year has nondigit characters
year_re = re.compile("\(\d\d\d\d")

In [4]:
# RATINGS FILE
file_path = "../../imdb/ratings.list"
with open(file_path, 'r') as f:
    l = None
    while(l != "MOVIE RATINGS REPORT"):
        l = f.readline().strip()
    #print(l)
    # empty line
    f.readline()
    # header
    f.readline()
    # Movies
    movies = []
    for m in f:
    #for i in range(100):
    #    m = f.readline()
        if m == "\n":
            break
        if m.find("(VG)") != -1: #skip video games
            continue            
        m=m.strip()
        try:
            m_list=m.split()
            rating_distribution = m_list[0]
            num_ratings = int(m_list[1])
            mean_rating = float(m_list[2])
            rest = " " . join(m_list[3:])
            if episode_re.search(rest) is None: # if not episode                
                year_match = year_re.search(rest)
                title = rest[:year_match.start()-1]
                year = int(year_match.group()[1:])
                movies.append(Movie(title, year, rating_distribution, num_ratings, mean_rating))
        except:
            #e = sys.exc_info()[0]
            #print( "<p>Error: %s</p>" % e )
            print("Failed to parse movie %s " %m)

Failed to parse movie 1......0.7      12   8.3  "By Any Means" (????) 
Failed to parse movie 1..1221..1       8   5.5  "Das große RTL Special zum Film" (????) {Evan Allmächtig} 
Failed to parse movie 0...0.0.06      12   8.5  "Das große RTL Special zum Film" (????) {Harry Potter und der Orden des Phönix} 
Failed to parse movie 200.0000.4      36   6.3  "Das große RTL Special zum Film" (????) {Harry Potter und die Kammer des Schreckens} 
Failed to parse movie 1....5.3..       6   5.8  "Germside" (????) 
Failed to parse movie ......46..       5   7.6  "Germspesyal" (????) 
Failed to parse movie ......22.6       5   9.0  "TV Rijnmond sport" (????) {Huldiging Feyenoord KNVB Beker 2008} 
Failed to parse movie 4...0.00.4      43   5.5  Passion (????/I) 
Failed to parse movie 1111000.01      26   4.3  The Colour of Life and Death (????) 


In [86]:
# GENRES FILE
file_path = "../../imdb/genres.list"
with open(file_path, 'r') as f:
    l = None
    while(l != "8: THE GENRES LIST"):
        l = f.readline().strip()
    #print(l)    
    # Dashes
    f.readline()
    # Empty
    f.readline()
    # Movies
    genres = {}
    for m in f:
        #break
    #for i in range(1000):
        #m = f.readline()
        if m == "\n":
            break
        if m.find("(VG)") != -1: #skip video games
            continue
        m=m.strip()

        try:
            m_list=m.split()
            #print(m_list)
            genre = m_list[-1]
            if episode_re.search(m) is None: # if not episode                
                year_match = year_re.search(m)
                title = m[:year_match.start()-1]
                year = year_match.group()[1:]
                #movies.append(Movie(title, year, rating_distribution, num_ratings, mean_rating))
                #print("%s \t %d \t %s" %(title, year, genre))
                tandy = title+year
                if tandy in genres: # FIX: TWO OR MORE TITLES IN SEPARATE YEARS, GETS MATCHED TO THE SAME GENRE
                    genres[tandy].append(genre)
                else:
                    genres[tandy]=[genre]
        except:
            pass
            #e = sys.exc_info()[0]
            #print( "<p>Error: %s</p>" % e )
            #print("Failed to parse movie %s " %m)

In [5]:
len(movies)

401448

In [88]:
len(genres)

1263141

In [83]:
count=0
for m in movies:
    if m.title+str(m.year) not in genres:
        count += 1
        #print(m.title)
        #break
    else:
        genre = genres[m.title+str(m.year)]        
        m.genre = genre
count

26099

In [70]:
m_ = movies
#m_=list(filter(lambda m: m.year>2000, m_))
m_=list(filter(lambda m: m.num_ratings>10000, m_))
m_=list(filter(lambda m: m.mean_rating>9, m_))

In [71]:
len(m_)

54

In [72]:
m_=sorted(m_, key=operator.attrgetter('num_ratings'), reverse=True)
#m_=sorted(m_, key=operator.attrgetter('year'), reverse=True)
#m_=sorted(m_, key=operator.attrgetter('mean_rating'), reverse=True)

In [73]:
m_[:20]

[The Shawshank Redemption	1994	0000000125	1809562	9.3,
 The Godfather	1972	0000000125	1235781	9.2,
 "Game of Thrones"	2011	0000000016	1152065	9.5,
 "Breaking Bad"	2008	0000000016	980194	9.5,
 "Sherlock"	2010	0000000025	576523	9.2,
 "True Detective"	2014	0000000134	358137	9.1,
 "Band of Brothers"	2001	0000000016	257721	9.5,
 "The Sopranos"	1999	0000000125	208181	9.2,
 "The Wire"	2002	0000000016	207006	9.4,
 "Firefly"	2002	0000000125	200624	9.1,
 "Avatar: The Last Airbender"	2005	0000000025	147898	9.2,
 "Rick and Morty"	2013	0000000026	121668	9.3,
 "Planet Earth"	2006	0000000016	117959	9.5,
 Dag II	2016	0000000009	88003	9.8,
 "Leyla ile Mecnun"	2011	0000000008	75011	9.2,
 "Cosmos: A Spacetime Odyssey"	2014	0000000026	71363	9.3,
 "Hagane no renkinjutsushi"	2009	0000000125	60351	9.1,
 "Last Week Tonight with John Oliver"	2014	0000000124	47961	9.1,
 "One Punch Man: Wanpanman"	2015	0000000125	45059	9.1,
 CM101MMXI Fundamentals	2013	0000000017	37183	9.3]

In [74]:
for m in m_:
    try:
       print(m.genre)
    except:
        print(m)

['Crime', 'Drama']
['Crime', 'Drama']
['Adventure', 'Drama', 'Fantasy', 'Romance']
['Crime', 'Drama', 'Thriller']
['Crime', 'Drama', 'Mystery']
['Crime', 'Drama', 'Mystery', 'Thriller']
['Action', 'Drama', 'History', 'War']
['Crime', 'Drama']
['Crime', 'Drama', 'Thriller']
['Adventure', 'Drama', 'Sci-Fi']
['Action', 'Adventure', 'Animation', 'Family', 'Fantasy', 'Mystery']
['Adventure', 'Animation', 'Comedy', 'Sci-Fi']
['Documentary']
['Drama', 'History', 'War']
['Adventure', 'Comedy', 'Drama', 'Fantasy', 'Mystery', 'Romance']
['Documentary']
['Action', 'Adventure', 'Animation', 'Comedy', 'Drama', 'Fantasy', 'Sci-Fi']
['Comedy', 'News', 'Talk-Show']
['Action', 'Animation', 'Comedy', 'Sci-Fi']
['Comedy', 'Documentary']
['Comedy', 'Drama']
['Action', 'Adventure', 'Comedy', 'Crime', 'Drama', 'Thriller']
['Action', 'Adventure', 'Drama', 'Horror', 'Thriller']
['Documentary']
['Comedy', 'Drama']
['Documentary']
['Comedy', 'Fantasy', 'Music', 'Short']
['Action', 'Adventure', 'Comedy', 'Crime'